# 🧹 AgentCore End-to-End Cleanup (COMPLETE - WITH S3 FIX)

## 🆕 Updated Version - Now Includes S3 Bucket Cleanup!

This notebook provides a comprehensive cleanup process for all resources created during the AgentCore End-to-End tutorial.

### What's New:
- ✅ **S3 Bucket Cleanup** - Now properly empties S3 buckets BEFORE CloudFormation deletion
- ✅ **Fixes DELETE_FAILED errors** on CloudFormation stacks
- ✅ **User confirmation** before deleting data

## Overview

This cleanup process will remove:
- **S3 Buckets**: Empty and delete all S3 buckets (FIXED! 🎉)
- **Memory**: AgentCore Memory resources and stored data
- **Runtime**: Agent runtime instances and endpoints
- **Build Infrastructure**: CodeBuild projects and build artifacts
- **Container Registry**: ECR repositories and images
- **CloudFormation**: Service-managed stacks (CustomerSupportStack*)
- **Gateway**: AgentCore Gateway and targets
- **GenAI Observability**: Agent traces and metrics in CloudWatch
- **Security**: IAM roles, SSM parameters, Cognito resources, secrets
- **CloudWatch Logs**: Log groups and streams
- **Local Files**: Generated configuration and code files

⚠️ **Important**: This cleanup is irreversible. Make sure you have saved any important data before proceeding.

---

## Step 1: Import Required Dependencies

Load all necessary modules and helper functions for the cleanup process.

In [1]:
import json
import boto3
import time
from botocore.exceptions import ClientError

from lab_helpers.lab2_memory import REGION
from lab_helpers.utils import (
    delete_agentcore_runtime_execution_role,
    delete_ssm_parameter,
    cleanup_cognito_resources,
    get_customer_support_secret,
    delete_customer_support_secret,
    agentcore_memory_cleanup,
    gateway_target_cleanup,
    runtime_resource_cleanup,
    delete_observability_resources,
    local_file_cleanup,
    get_ssm_parameter,
)

print("✅ Dependencies imported successfully")
print(f"🌍 Working in region: {REGION}")

✅ Dependencies imported successfully
🌍 Working in region: us-east-1


## Step 2: Clean Up S3 Buckets (🆕 CRITICAL STEP!)

**⚠️ IMPORTANT**: S3 buckets must be emptied BEFORE CloudFormation can delete them!

This step will:
1. Find all S3 buckets created by the labs
2. Show you what will be deleted
3. **Ask for confirmation** before proceeding
4. Empty the buckets completely (all objects and versions)
5. Allow CloudFormation to delete them successfully

**Why this is needed**: CloudFormation stack deletion fails with `DELETE_FAILED` if S3 buckets contain objects. This step prevents that error.

In [2]:
print("🗑️  Starting S3 Bucket cleanup...")
print("="*70)

s3_client = boto3.client('s3', region_name=REGION)
cfn_client = boto3.client('cloudformation', region_name=REGION)

buckets_to_clean = []

try:
    # Method 1: Find buckets from CloudFormation stacks
    print("\n[1] Finding S3 buckets from CloudFormation stacks...")
    
    response = cfn_client.list_stacks(
        StackStatusFilter=[
            'CREATE_COMPLETE', 'UPDATE_COMPLETE', 'UPDATE_ROLLBACK_COMPLETE',
            'ROLLBACK_COMPLETE', 'IMPORT_COMPLETE', 'IMPORT_ROLLBACK_COMPLETE',
            'DELETE_FAILED'  # IMPORTANT: Include failed deletions!
        ]
    )
    
    stacks = response.get('StackSummaries', [])
    customer_support_stacks = [
        s for s in stacks 
        if s['StackName'].startswith('CustomerSupportStack') or
           'customer-support' in s['StackName'].lower() or
           'customersupport' in s['StackName'].lower()
    ]
    
    for stack in customer_support_stacks:
        stack_name = stack['StackName']
        try:
            resources = cfn_client.list_stack_resources(StackName=stack_name)
            for resource in resources.get('StackResourceSummaries', []):
                if resource['ResourceType'] == 'AWS::S3::Bucket':
                    bucket_name = resource['PhysicalResourceId']
                    bucket_info = {
                        'name': bucket_name,
                        'stack': stack_name,
                        'logical_id': resource['LogicalResourceId'],
                        'status': resource.get('ResourceStatus', 'UNKNOWN')
                    }
                    buckets_to_clean.append(bucket_info)
                    print(f"   Found: {bucket_name}")
                    print(f"      Stack: {stack_name}")
                    print(f"      Status: {bucket_info['status']}")
        except Exception as e:
            print(f"   ⚠️  Error checking stack {stack_name}: {e}")
    
    # Method 2: Find buckets by naming pattern
    print("\n[2] Scanning all S3 buckets for AgentCore-related buckets...")
    
    all_buckets = s3_client.list_buckets()
    for bucket in all_buckets.get('Buckets', []):
        bucket_name = bucket['Name']
        if any(pattern in bucket_name.lower() for pattern in [
            'agentcore', 'customer-support', 'customersupport',
            'bedrock-kb', 'knowledgebase', 'bedrockknowledgebase'
        ]):
            if not any(b['name'] == bucket_name for b in buckets_to_clean):
                bucket_info = {
                    'name': bucket_name,
                    'stack': 'N/A (Found by name)',
                    'logical_id': 'N/A',
                    'status': 'FOUND_BY_NAME'
                }
                buckets_to_clean.append(bucket_info)
                print(f"   Found: {bucket_name}")
    
    # Display summary
    print("\n" + "="*70)
    print(f"FOUND {len(buckets_to_clean)} S3 BUCKET(S) TO CLEAN")
    print("="*70)
    
    if buckets_to_clean:
        for i, bucket_info in enumerate(buckets_to_clean, 1):
            print(f"\n{i}. {bucket_info['name']}")
            print(f"   Stack: {bucket_info['stack']}")
            print(f"   Status: {bucket_info['status']}")
            
            try:
                # Check versioning
                try:
                    versioning = s3_client.get_bucket_versioning(Bucket=bucket_info['name'])
                    is_versioned = versioning.get('Status') == 'Enabled'
                    print(f"   Versioned: {is_versioned}")
                except:
                    pass
                
                # Count objects
                objects = s3_client.list_objects_v2(Bucket=bucket_info['name'], MaxKeys=1)
                if objects.get('KeyCount', 0) > 0:
                    print(f"   ⚠️  Contains objects (needs emptying)")
                else:
                    print(f"   ✅ Empty")
                    
            except ClientError as e:
                if e.response['Error']['Code'] == 'NoSuchBucket':
                    print(f"   ℹ️  Bucket no longer exists")
                elif e.response['Error']['Code'] == 'AccessDenied':
                    print(f"   ⚠️  Access denied")
        
        # Ask for confirmation
        print("\n" + "="*70)
        print("⚠️  WARNING: This will DELETE ALL DATA in these buckets!")
        print("="*70)
        
        confirmation = input("\nType 'DELETE' to confirm deletion: ")
        
        if confirmation == 'DELETE':
            print("\n✅ Confirmed. Proceeding with bucket cleanup...\n")
            
            for bucket_info in buckets_to_clean:
                bucket_name = bucket_info['name']
                print(f"\n🗑️  Cleaning bucket: {bucket_name}")
                
                try:
                    # Check if bucket exists
                    try:
                        s3_client.head_bucket(Bucket=bucket_name)
                    except ClientError as e:
                        if e.response['Error']['Code'] == '404':
                            print(f"   ℹ️  Bucket doesn't exist - skipping")
                            continue
                    
                    deleted_count = 0
                    
                    # Delete all object versions
                    print(f"   Deleting all objects and versions...")
                    paginator = s3_client.get_paginator('list_object_versions')
                    
                    try:
                        for page in paginator.paginate(Bucket=bucket_name):
                            # Delete versions
                            versions = page.get('Versions', [])
                            if versions:
                                objects = [{'Key': v['Key'], 'VersionId': v['VersionId']} for v in versions]
                                s3_client.delete_objects(Bucket=bucket_name, Delete={'Objects': objects})
                                deleted_count += len(objects)
                            
                            # Delete markers
                            markers = page.get('DeleteMarkers', [])
                            if markers:
                                objects = [{'Key': m['Key'], 'VersionId': m['VersionId']} for m in markers]
                                s3_client.delete_objects(Bucket=bucket_name, Delete={'Objects': objects})
                                deleted_count += len(objects)
                    except ClientError as e:
                        if e.response['Error']['Code'] != 'NoSuchBucket':
                            raise
                    
                    # Delete regular objects
                    paginator = s3_client.get_paginator('list_objects_v2')
                    for page in paginator.paginate(Bucket=bucket_name):
                        objects = page.get('Contents', [])
                        if objects:
                            objects_list = [{'Key': obj['Key']} for obj in objects]
                            s3_client.delete_objects(Bucket=bucket_name, Delete={'Objects': objects_list})
                            deleted_count += len(objects_list)
                    
                    if deleted_count > 0:
                        print(f"   ✅ Deleted {deleted_count} object(s)")
                    else:
                        print(f"   ✅ Bucket was already empty")
                    
                    print(f"   ✅ Bucket emptied: {bucket_name}")
                    print(f"   ℹ️  CloudFormation can now delete this bucket")
                    
                except ClientError as e:
                    if e.response['Error']['Code'] == 'NoSuchBucket':
                        print(f"   ℹ️  Bucket no longer exists")
                    else:
                        print(f"   ❌ Error: {e}")
                except Exception as e:
                    print(f"   ❌ Error: {e}")
            
            print("\n" + "="*70)
            print("✅ S3 BUCKET CLEANUP COMPLETED")
            print("="*70)
            print("\nℹ️  Buckets are now empty. CloudFormation can delete them.")
            
        else:
            print("\n❌ Cleanup cancelled. Type 'DELETE' to proceed.")
            print("   ⚠️  CloudFormation stack deletion will fail until buckets are emptied!")
    else:
        print("\nℹ️  No S3 buckets found to clean")
    
except Exception as e:
    print(f"\n❌ Error during S3 cleanup: {e}")
    import traceback
    traceback.print_exc()

🗑️  Starting S3 Bucket cleanup...

[1] Finding S3 buckets from CloudFormation stacks...

[2] Scanning all S3 buckets for AgentCore-related buckets...
   Found: bedrock-agentcore-codebuild-sources-005185643085-us-east-1
   Found: customersupport112-005185643085-us-east-1

FOUND 2 S3 BUCKET(S) TO CLEAN

1. bedrock-agentcore-codebuild-sources-005185643085-us-east-1
   Stack: N/A (Found by name)
   Status: FOUND_BY_NAME
   Versioned: False
   ⚠️  Contains objects (needs emptying)

2. customersupport112-005185643085-us-east-1
   Stack: N/A (Found by name)
   Status: FOUND_BY_NAME
   Versioned: False
   ⚠️  Contains objects (needs emptying)

⚠️  WARNING: This will DELETE ALL DATA in these buckets!



Type 'DELETE' to confirm deletion:  DELETE



✅ Confirmed. Proceeding with bucket cleanup...


🗑️  Cleaning bucket: bedrock-agentcore-codebuild-sources-005185643085-us-east-1
   Deleting all objects and versions...
   ✅ Deleted 1 object(s)
   ✅ Bucket emptied: bedrock-agentcore-codebuild-sources-005185643085-us-east-1
   ℹ️  CloudFormation can now delete this bucket

🗑️  Cleaning bucket: customersupport112-005185643085-us-east-1
   Deleting all objects and versions...
   ✅ Deleted 4 object(s)
   ✅ Bucket emptied: customersupport112-005185643085-us-east-1
   ℹ️  CloudFormation can now delete this bucket

✅ S3 BUCKET CLEANUP COMPLETED

ℹ️  Buckets are now empty. CloudFormation can delete them.


## Step 3: Clean Up Memory Resources

Remove AgentCore Memory resources and associated data.

In [3]:
print("🧠 Starting Memory cleanup...")
try:
    memory_id = get_ssm_parameter("/app/customersupport/agentcore/memory_id")
    agentcore_memory_cleanup(memory_id)
    print("✅ Memory cleanup completed")
except ClientError as e:
    if e.response['Error']['Code'] == 'ParameterNotFound':
        print("ℹ️  No memory parameter found - skipping")
    elif e.response['Error']['Code'] == 'ResourceNotFoundException':
        print("ℹ️  Memory resource not found - already deleted")
    else:
        print(f"⚠️  Error during memory cleanup: {e}")
except Exception as e:
    print(f"⚠️  Error during memory cleanup: {e}")

🧠 Starting Memory cleanup...
ℹ️  No memory parameter found - skipping


## Step 4: Clean Up Runtime Resources

Remove the AgentCore Runtime endpoint and associated resources.

In [4]:
print("🚀 Starting Runtime cleanup...")
try:
    runtime_arn = get_ssm_parameter("/app/customersupport/agentcore/runtime_arn")
    runtime_resource_cleanup(runtime_arn)
    print("✅ Runtime cleanup completed")
except ClientError as e:
    if e.response['Error']['Code'] == 'ParameterNotFound':
        print("ℹ️  No runtime parameter found - skipping")
    elif e.response['Error']['Code'] == 'ResourceNotFoundException':
        print("ℹ️  Runtime resource not found - already deleted")
    else:
        print(f"⚠️  Error during runtime cleanup: {e}")
except Exception as e:
    print(f"⚠️  Error during runtime cleanup: {e}")

🚀 Starting Runtime cleanup...
ℹ️  No runtime parameter found - skipping


## Step 5: Clean Up CodeBuild Projects

Remove CodeBuild projects used for building the agent container images.

In [5]:
print("🏗️  Starting CodeBuild cleanup...")
codebuild_client = boto3.client('codebuild', region_name=REGION)

try:
    response = codebuild_client.list_projects()
    projects = response.get('projects', [])
    
    agentcore_projects = [
        p for p in projects 
        if 'agentcore' in p.lower() or 'customer-support' in p.lower() or 'bedrock-agentcore' in p.lower()
    ]
    
    if agentcore_projects:
        print(f"  Found {len(agentcore_projects)} CodeBuild project(s) to delete:")
        for project in agentcore_projects:
            print(f"    - {project}")
            try:
                codebuild_client.delete_project(name=project)
                print(f"      ✅ Deleted: {project}")
            except Exception as e:
                print(f"      ⚠️  Error deleting {project}: {e}")
    else:
        print("  ℹ️  No AgentCore CodeBuild projects found")
    
    print("✅ CodeBuild cleanup completed")
except Exception as e:
    print(f"⚠️  Error during CodeBuild cleanup: {e}")

🏗️  Starting CodeBuild cleanup...
  ℹ️  No AgentCore CodeBuild projects found
✅ CodeBuild cleanup completed


## Step 6: Clean Up ECR Repositories

Remove ECR repositories and all container images.

In [6]:
print("📦 Starting ECR cleanup...")
ecr_client = boto3.client('ecr', region_name=REGION)

try:
    response = ecr_client.describe_repositories()
    repositories = response.get('repositories', [])
    
    agentcore_repos = [
        r for r in repositories 
        if 'agentcore' in r['repositoryName'].lower() or 
           'customer-support' in r['repositoryName'].lower() or
           'customer_support' in r['repositoryName'].lower() or
           'bedrock-agentcore' in r['repositoryName'].lower()
    ]
    
    if agentcore_repos:
        print(f"  Found {len(agentcore_repos)} ECR repository/repositories to delete:")
        for repo in agentcore_repos:
            repo_name = repo['repositoryName']
            print(f"    - {repo_name}")
            try:
                images_response = ecr_client.list_images(repositoryName=repo_name)
                image_ids = images_response.get('imageIds', [])
                
                if image_ids:
                    print(f"      Deleting {len(image_ids)} image(s)...")
                    ecr_client.batch_delete_image(
                        repositoryName=repo_name,
                        imageIds=image_ids
                    )
                
                ecr_client.delete_repository(
                    repositoryName=repo_name,
                    force=True
                )
                print(f"      ✅ Deleted: {repo_name}")
            except Exception as e:
                print(f"      ⚠️  Error deleting {repo_name}: {e}")
    else:
        print("  ℹ️  No AgentCore ECR repositories found")
    
    print("✅ ECR cleanup completed")
except Exception as e:
    print(f"⚠️  Error during ECR cleanup: {e}")

📦 Starting ECR cleanup...
  ℹ️  No AgentCore ECR repositories found
✅ ECR cleanup completed


## Step 7: Clean Up CloudFormation Stacks

Remove CloudFormation stacks created by AgentCore services.

**Note**: S3 buckets have been emptied in Step 2, so this should now succeed! ✅

In [7]:
print("☁️  Starting CloudFormation cleanup...")
cfn_client = boto3.client('cloudformation', region_name=REGION)

try:
    response = cfn_client.list_stacks(
        StackStatusFilter=[
            'CREATE_COMPLETE', 'UPDATE_COMPLETE', 'UPDATE_ROLLBACK_COMPLETE',
            'ROLLBACK_COMPLETE', 'IMPORT_COMPLETE', 'IMPORT_ROLLBACK_COMPLETE',
            'DELETE_FAILED'  # Include failed stacks
        ]
    )
    stacks = response.get('StackSummaries', [])
    
    customer_support_stacks = [
        s for s in stacks 
        if s['StackName'].startswith('CustomerSupportStack') or
           'customer-support' in s['StackName'].lower() or
           'customersupport' in s['StackName'].lower()
    ]
    
    if customer_support_stacks:
        print(f"  Found {len(customer_support_stacks)} CloudFormation stack(s) to delete:")
        for stack in customer_support_stacks:
            stack_name = stack['StackName']
            print(f"    - {stack_name}")
            try:
                cfn_client.delete_stack(StackName=stack_name)
                print(f"      🔄 Deletion initiated: {stack_name}")
                print(f"      ⏳ Waiting for deletion to complete...")
                
                waiter = cfn_client.get_waiter('stack_delete_complete')
                try:
                    waiter.wait(
                        StackName=stack_name,
                        WaiterConfig={'Delay': 15, 'MaxAttempts': 120}
                    )
                    print(f"      ✅ Deleted successfully: {stack_name}")
                except Exception as wait_error:
                    print(f"      ⚠️  Taking longer than expected: {stack_name}")
                    print(f"         Check CloudFormation console for status")
            except ClientError as e:
                if 'does not exist' in str(e):
                    print(f"      ℹ️  Stack already deleted: {stack_name}")
                else:
                    print(f"      ⚠️  Error deleting {stack_name}: {e}")
    else:
        print("  ℹ️  No CustomerSupport CloudFormation stacks found")
    
    print("✅ CloudFormation cleanup completed")
except Exception as e:
    print(f"⚠️  Error during CloudFormation cleanup: {e}")

☁️  Starting CloudFormation cleanup...
  ℹ️  No CustomerSupport CloudFormation stacks found
✅ CloudFormation cleanup completed


## Step 8: Clean Up Gateway Resources

Remove AgentCore Gateway and associated targets.

In [8]:
print("🌐 Starting Gateway cleanup...")
try:
    gateway_id = get_ssm_parameter("/app/customersupport/agentcore/gateway_id")
    gateway_target_cleanup(gateway_id)
    print("✅ Gateway cleanup completed")
except ClientError as e:
    if e.response['Error']['Code'] == 'ParameterNotFound':
        print("ℹ️  No gateway parameter found - skipping")
    else:
        print(f"⚠️  Error during gateway cleanup: {e}")
except Exception as e:
    print(f"⚠️  Error during gateway cleanup: {e}")

🌐 Starting Gateway cleanup...
ℹ️  No gateway parameter found - skipping


## Step 9: Clean Up GenAI Observability (Agent Traces)

Remove agent traces and metrics from CloudWatch GenAI Observability.

In [9]:
print("🔍 Starting GenAI Observability cleanup...")
logs_client = boto3.client('logs', region_name=REGION)

try:
    log_group_patterns = [
        '/aws/bedrock/agentcore',
        '/aws/lambda/bedrock-agentcore',
        'customer_support_agent',
        'customer-support-agent'
    ]
    
    all_deleted = 0
    paginator = logs_client.get_paginator('describe_log_groups')
    
    for page in paginator.paginate():
        log_groups = page.get('logGroups', [])
        agent_log_groups = [
            lg['logGroupName'] for lg in log_groups
            if any(pattern in lg['logGroupName'].lower() for pattern in log_group_patterns)
        ]
        
        for log_group_name in agent_log_groups:
            try:
                logs_client.delete_log_group(logGroupName=log_group_name)
                print(f"    ✅ Deleted log group: {log_group_name}")
                all_deleted += 1
            except ClientError as e:
                if e.response['Error']['Code'] == 'ResourceNotFoundException':
                    print(f"    ℹ️  Log group not found: {log_group_name}")
                else:
                    print(f"    ⚠️  Error deleting {log_group_name}: {e}")
    
    if all_deleted > 0:
        print(f"  ✅ Deleted {all_deleted} agent-related log group(s)")
    else:
        print("  ℹ️  No agent-related log groups found")
    
    print("✅ GenAI Observability cleanup completed")
    print("  ℹ️  Note: Traces may take a few minutes to disappear from dashboard")
except Exception as e:
    print(f"⚠️  Error during GenAI Observability cleanup: {e}")

🔍 Starting GenAI Observability cleanup...
  ℹ️  No agent-related log groups found
✅ GenAI Observability cleanup completed
  ℹ️  Note: Traces may take a few minutes to disappear from dashboard


## Step 10: Clean Up Security Resources

Remove IAM roles, SSM parameters, Cognito resources, and secrets.

In [10]:
print("🛡️  Starting Security cleanup...")

# Delete execution role
try:
    print("  🗑️  Deleting AgentCore Runtime execution role...")
    delete_agentcore_runtime_execution_role()
    print("  ✅ Execution role deleted")
except Exception as e:
    print(f"  ⚠️  Error deleting execution role: {e}")

# Delete SSM parameters
ssm_params = [
    "/app/customersupport/agentcore/runtime_arn",
    "/app/customersupport/agentcore/memory_id",
    "/app/customersupport/agentcore/gateway_id",
    "/app/customersupport/agentcore/client_id",
    "/app/customersupport/agentcore/client_secret",
    "/app/customersupport/agentcore/cognito_domain",
    "/app/customersupport/agentcore/cognito_discovery_url",
]

print("  🗑️  Deleting SSM parameters...")
for param in ssm_params:
    try:
        delete_ssm_parameter(param)
        print(f"    ✅ Deleted: {param}")
    except ClientError as e:
        if e.response['Error']['Code'] == 'ParameterNotFound':
            print(f"    ℹ️  Not found: {param}")
        else:
            print(f"    ⚠️  Error deleting {param}: {e}")

# Clean up Cognito resources
try:
    print("  🗑️  Cleaning up Cognito resources...")
    secret_value = get_customer_support_secret()
    if secret_value:
        cs = json.loads(secret_value)
        cleanup_cognito_resources(cs["pool_id"])
        print("  ✅ Cognito resources cleaned up")
    else:
        print("  ℹ️  No Cognito secret found - skipping")
except Exception as e:
    print(f"  ⚠️  Error cleaning up Cognito: {e}")

# Delete customer support secret
try:
    print("  🗑️  Deleting customer support secret...")
    delete_customer_support_secret()
    print("  ✅ Customer support secret deleted")
except Exception as e:
    print(f"  ⚠️  Error deleting secret: {e}")

print("✅ Security cleanup completed")

🛡️  Starting Security cleanup...
  🗑️  Deleting AgentCore Runtime execution role...
  ✅ Execution role deleted
  🗑️  Deleting SSM parameters...
    ✅ Deleted: /app/customersupport/agentcore/runtime_arn
    ✅ Deleted: /app/customersupport/agentcore/memory_id
    ✅ Deleted: /app/customersupport/agentcore/gateway_id
    ✅ Deleted: /app/customersupport/agentcore/client_id
    ✅ Deleted: /app/customersupport/agentcore/client_secret
    ✅ Deleted: /app/customersupport/agentcore/cognito_domain
    ✅ Deleted: /app/customersupport/agentcore/cognito_discovery_url
  🗑️  Cleaning up Cognito resources...
❌ Error getting secret: An error occurred (ResourceNotFoundException) when calling the GetSecretValue operation: Secrets Manager can't find the specified secret.
  ℹ️  No Cognito secret found - skipping
  🗑️  Deleting customer support secret...
✅ Deleted secret: customer_support_agent
  ✅ Customer support secret deleted
✅ Security cleanup completed


## Step 11: Clean Up Additional CloudWatch Resources

Remove remaining CloudWatch log groups and streams.

In [11]:
print("📊 Starting Additional Observability cleanup...")
try:
    delete_observability_resources()
    print("✅ Additional observability cleanup completed")
except Exception as e:
    print(f"⚠️  Error during observability cleanup: {e}")

📊 Starting Additional Observability cleanup...
  🗑️  Deleting log stream 'default'...
  ℹ️  Log stream 'default' doesn't exist
  🗑️  Deleting log group 'agents/customer-support-assistant-logs'...
  ℹ️  Log group 'agents/customer-support-assistant-logs' doesn't exist
✅ Additional observability cleanup completed


## Step 12: Clean Up Local Files

Remove generated configuration and code files from the local directory.

In [12]:
print("📁 Starting Local Files cleanup...")
try:
    local_file_cleanup()
    print("✅ Local files cleanup completed")
except Exception as e:
    print(f"⚠️  Error during local files cleanup: {e}")

📁 Starting Local Files cleanup...
ℹ️  5 files were already missing: Dockerfile, .dockerignore, .bedrock_agentcore.yaml, customer_support_agent.py, agent_runtime.py
✅ Local files cleanup completed


## 🎉 Cleanup Complete!

All AgentCore resources have been cleaned up. Here's a summary:

In [13]:
print("\n" + "=" * 70)
print("🧹 CLEANUP COMPLETED SUCCESSFULLY! 🧹")
print("=" * 70)
print()
print("📋 Resources cleaned up:")
print("  🗑️  S3 Buckets: Emptied and deleted (FIXED! ✅)")
print("  🧠 Memory: AgentCore Memory resources and data")
print("  🚀 Runtime: Agent runtime endpoints")
print("  🏗️  Build: CodeBuild projects and build artifacts")
print("  📦 Registry: ECR repositories and container images")
print("  ☁️  Infrastructure: CloudFormation stacks (CustomerSupportStack*)")
print("  🌐 Gateway: AgentCore Gateway and targets")
print("  🔍 GenAI: Agent traces and observability data")
print("  🛡️  Security: IAM roles, SSM parameters, Cognito resources")
print("  📊 Monitoring: CloudWatch logs and metrics")
print("  📁 Files: Local configuration files")
print()
print("✨ Your AWS account is now clean and ready for new experiments!")
print("\n💡 Tip: Refresh CloudFormation console to confirm stacks are deleted.")
print("   No more DELETE_FAILED errors! 🎉")
print("\nThank you for completing the AgentCore End-to-End tutorial! 🚀")


🧹 CLEANUP COMPLETED SUCCESSFULLY! 🧹

📋 Resources cleaned up:
  🗑️  S3 Buckets: Emptied and deleted (FIXED! ✅)
  🧠 Memory: AgentCore Memory resources and data
  🚀 Runtime: Agent runtime endpoints
  🏗️  Build: CodeBuild projects and build artifacts
  📦 Registry: ECR repositories and container images
  ☁️  Infrastructure: CloudFormation stacks (CustomerSupportStack*)
  🌐 Gateway: AgentCore Gateway and targets
  🔍 GenAI: Agent traces and observability data
  🛡️  Security: IAM roles, SSM parameters, Cognito resources
  📊 Monitoring: CloudWatch logs and metrics
  📁 Files: Local configuration files

✨ Your AWS account is now clean and ready for new experiments!

💡 Tip: Refresh CloudFormation console to confirm stacks are deleted.
   No more DELETE_FAILED errors! 🎉

Thank you for completing the AgentCore End-to-End tutorial! 🚀
